In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [5]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 5e-25
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.55, 1.6, 1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.5, 0.6, 0.7, 0.9)]
o co2 band3c
  atmpro = mls
  band = 3c
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 800
  vmin = 720
  w_diffuse = [(1.7, 1.6, 1.8), (1.8, 1.7), (1.5, 1.6, 1.7, 1.8)]
  wgt = [(0.6, 0.4, 0.7), (0.7, 0.4), (0.3, 0.4, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.253565                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.099289
0.000750      2.0  0.253741                  2.0  0.103167
0.001052      3.0  0.264521                  3.0  0.112958
0.001476      4.0  0.274320                  4.0  0.122801
0.002070      5.0  0.282196                  5.0  0.132444
0.002904      6.0  0.287747                  6.0  0.141642
0.004074      7.0  0.290000                  7.0  0.150026
0.005714      8.0  0.288125                  8.0  0.157211
0.008015      9.0  0.280969                  9.0  0.162422
0.011243     10.0  0.267291                 10.0  0.164819
0.015771     11.0  0.245779                 11.0  0.163030
0.022122     12.0  0.214917                 12.0  0.155122
0.031031     13.0  0.172816                 13.0  0.138669
0.043528     14.0  0.120400                 14.0  0.113704
0.061057     15.0  0.101435                 15.0  0.115976
0.085645     16.0  0.139053                 16.0  0.167081
0.120136     17.0  0.189440                 17.0  0.227910
0.168516     18.0  0.226890                 18.0  0.263752
0.236378     19.0  0.258186                 19.0  0.271190
0.331549     20.0  0.295526                 20.0  0.271153
0.465100     21.0  0.361773                 21.0  0.311834
0.652400     22.0  0.476333                 22.0  0.432549
0.915100     23.0  0.483616                 23.0  0.465012
1.283650     24.0  0.403821                 24.0  0.407712
1.800600     25.0  0.338784                 25.0  0.350258
2.525700     26.0  0.268299                 26.0  0.279869
3.542800     27.0  0.213223                 27.0  0.228616
4.969550     28.0  0.171281                 28.0  0.191261
6.970850     29.0  0.136919                 29.0  0.158965
9.778100     30.0  0.107628                 30.0  0.124172
13.715850    31.0  0.082373                 31.0  0.090060
19.239350    32.0  0.060832                 32.0  0.059929
26.987250    33.0  0.043207                 33.0  0.036600
37.855300    34.0  0.030233                 34.0  0.023168
53.100050    35.0  0.012333                 35.0  0.008617
73.887500    36.0 -0.008562                 36.0 -0.010227
97.662500    37.0 -0.021921                 37.0 -0.023862
121.437500   38.0 -0.020843                 38.0 -0.023406
145.212500   39.0 -0.010451                 39.0 -0.013733
168.987500   40.0 -0.004142                 40.0 -0.007525
192.762500   41.0  0.002087                 41.0 -0.000892
216.537500   42.0  0.008626                 42.0  0.005790
240.312500   43.0  0.015568                 43.0  0.012466
264.087500   44.0  0.023001                 44.0  0.019118
287.862500   45.0  0.031047                 45.0  0.026037
311.637500   46.0  0.039816                 46.0  0.033514
335.412500   47.0  0.049340                 47.0  0.041828
359.187500   48.0  0.059396                 48.0  0.051130
382.962500   49.0  0.069099                 49.0  0.060655
406.737500   50.0  0.078963                 50.0  0.070651
430.512500   51.0  0.088914                 51.0  0.080893
454.287500   52.0  0.098894                 52.0  0.091349
478.062500   53.0  0.108865                 53.0  0.101950
501.837500   54.0  0.118759                 54.0  0.112825
525.612500   55.0  0.128643                 55.0  0.123942
549.387500   56.0  0.138468                 56.0  0.135706
573.162500   57.0  0.148299                 57.0  0.147837
596.937500   58.0  0.158274                 58.0  0.160045
620.712500   59.0  0.168471                 59.0  0.172084
644.487500   60.0  0.179157                 60.0  0.184064
668.262500   61.0  0.190096                 61.0  0.196009
692.037500   62.0  0.199786                 62.0  0.206761
715.812500   63.0  0.205715                 63.0  0.214014
739.587500   64.0  0.212600                 64.0  0.222841
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -26.007355   0.000000 -26.007355        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -25.691664   0.000001   
6.244000e-04 2     -26.007370   0.000034 -26.007336 -25.691681   0.000025   
8.759000e-04 3     -26.007376   0.000048 -26.007328 -25.691688   0.000035   
1.228600e-03 4     -26.007385   0.000068 -26.007317 -25.691697   0.000049   
1.723400e-03 5     -26.007397   0.000096 -26.007301 -25.691711   0.000070   
2.417400e-03 6     -26.007413   0.000135 -26.007278 -25.691729   0.000099   
3.390900e-03 7     -26.007436   0.000191 -26.007245 -25.691754   0.000140   
4.756500e-03 8     -26.007467   0.000269 -26.007198 -25.691789   0.000200   
6.672000e-03 9     -26.007511   0.000378 -26.007133 -25.691838   0.000285   
9.358900e-03 10    -26.007571   0.000528 -26.007043 -25.691907   0.000405   
1.312780e-02 11    -26.007656   0.000732 -26.006924 -25.692004   0.000576   
1.841450e-02 12    -26.007777   0.001007 -26.006770 -25.692141   0.000815   
2.583020e-02 13    -26.007949   0.001368 -26.006581 -25.692337   0.001147   
3.623230e-02 14    -26.008203   0.001834 -26.006368 -25.692621   0.001602   
5.082340e-02 15    -26.008587   0.002427 -26.006160 -25.693040   0.002218   
7.129060e-02 16    -26.009159   0.003245 -26.005914 -25.693646   0.003105   
1.000000e-01 17    -26.009951   0.004510 -26.005441 -25.694471   0.004498   
1.402710e-01 18    -26.010993   0.006455 -26.004538 -25.695541   0.006655   
1.967600e-01 19    -26.012318   0.009298 -26.003019 -25.696879   0.009758   
2.759970e-01 20    -26.013922   0.013326 -26.000596 -25.698470   0.013895   
3.871000e-01 21    -26.015682   0.018975 -25.996707 -25.700183   0.019177   
5.431000e-01 22    -26.017045   0.027024 -25.990021 -25.701474   0.026231   
7.617000e-01 23    -26.016161   0.038474 -25.977687 -25.700401   0.036360   
1.068500e+00 24    -26.012293   0.052182 -25.960111 -25.696187   0.049047   
1.498800e+00 25    -26.006452   0.066925 -25.939527 -25.689830   0.063475   
2.102400e+00 26    -25.998775   0.083471 -25.915304 -25.681399   0.080090   
2.949000e+00 27    -25.990002   0.101604 -25.888397 -25.671571   0.098332   
4.136600e+00 28    -25.980343   0.121942 -25.858401 -25.660391   0.119316   
5.802500e+00 29    -25.969744   0.145143 -25.824601 -25.647662   0.144334   
8.139200e+00 30    -25.958264   0.171562 -25.786702 -25.633182   0.173860   
1.141700e+01 31    -25.946157   0.201245 -25.744912 -25.617138   0.206035   
1.601470e+01 32    -25.933976   0.233927 -25.700049 -25.600116   0.238067   
2.246400e+01 33    -25.922707   0.269131 -25.653576 -25.583486   0.267226   
3.151050e+01 34    -25.913668   0.306394 -25.607274 -25.569181   0.292147   
4.420010e+01 35    -25.906755   0.344927 -25.561828 -25.557318   0.315113   
6.200000e+01 36    -25.911626   0.375801 -25.535825 -25.559956   0.335920   
8.577500e+01 37    -25.952730   0.392792 -25.559937 -25.606474   0.353632   
1.095500e+02 38    -26.031550   0.409876 -25.621675 -25.696781   0.376729   
1.333250e+02 39    -26.142025   0.461651 -25.680374 -25.821733   0.435755   
1.571000e+02 40    -26.271357   0.561549 -25.709808 -25.965960   0.541302   
1.808750e+02 41    -26.417654   0.696180 -25.721474 -26.126720   0.680869   
2.046500e+02 42    -26.579299   0.863702 -25.715597 -26.302783   0.854419   
2.284250e+02 43    -26.755006   1.063704 -25.691302 -26.494001   1.061945   
2.522000e+02 44    -26.943519   1.296061 -25.647457 -26.698590   1.301646   
2.759750e+02 45    -27.143906   1.561226 -25.582679 -26.915139   1.572042   
2.997500e+02 46    -27.355483   1.860242 -25.495240 -27.142721   1.872960   
3.235250e+02 47    -27.577778   2.194673 -25.383105 -27.380526   2.205162   
3.473000e+02 48    -27.810506   2.566359 -25.244147 -27.627917   2.570366   
3.710750e+02 49    -28

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')